In [2]:
#import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.model_selection import train_test_split
import warnings
from sklearn_porter import Porter
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# We are reading our data
df = pd.read_csv("heart.csv")

In [4]:
#process dataset
a = pd.get_dummies(df['cp'], prefix = "cp")
frames = [df, a]
df = pd.concat(frames, axis = 1)
df = df.drop(columns = ['cp', 'restecg', 'oldpeak', 'slope', 'ca', 'thal'])

In [5]:
#seperate data and target
y = df.target.values
x_data = df.drop(['target'], axis = 1)

In [6]:
# Normalize
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [7]:
#prepare for train test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=9478)

In [ ]:
#call model
gnb = gnb()
gnb.fit(x_train,y_train)

In [44]:
def prepare_data_form(data):
    data = np.array(data.split(","),dtype=np.float64)
    a = np.zeros(4,dtype=np.float64)
    for i in range(4):
        if data[2]==i:
            a[i]=1
    data = np.delete(data,2)
    data = np.insert(data,len(data),a)
    data = data.reshape(1,-1)
    new_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return data

In [45]:
accuracies = {}
acc = gnb.score(x_train,y_train)*100
accuracies['GaussianNB'] = acc
print("Train Accuracy {:.2f}%".format(acc))

Train Accuracy 75.62%


In [46]:
acc2 = gnb.score(x_test,y_test)*100
accuracies['GaussianNB'] = acc2
print("Test Accuracy {:.2f}%".format(acc2))

Test Accuracy 93.44%


result = []
result2 = []
for i in range(10000):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=i)
    x_train = x_train.T
    y_train = y_train.T
    x_test = x_test.T
    y_test = y_test.T
    accuracies = {}
    ml = gnb()
    ml.fit(x_train.T,y_train.T)
    acc = ml.score(x_train.T,y_train.T)*100
    acc2 = ml.score(x_test.T,y_test.T)*100
    result.append(acc+acc2)
print(max(result),result.index(max(result)))

In [47]:
form1 = "63,1,3,145,233,1,150,0" #1
form11 = "0.708333,1,0.481132,0.244292,1,0.603053,0,0,0,0,1" #1
form0 = "0.166667,1,0.339623,0.283105,0,0.885496,0,0,0,1,0" #1
form2 = "58,1,0,114,318,0,140,0" #0
form22 = "0.604167,1,0.188679,0.438356,0,0.526718,0,1,0,0,0" #0

form = np.array(form0.split(","),dtype=np.float64)
form = form.reshape(1,-1)

form_norm = prepare_data_form(form2)

result = "{:.2f}%".format(gnb.predict_proba(form)[0,1]*100)

print(result)
print(gnb.predict_proba(form))

99.73%
[[0.00267896 0.99732104]]


In [48]:
porter = Porter(gnb, language='java')
output = porter.export(embed_data=True)
print(output)

class GaussianNB {

    private double[] priors;
    private double[][] sigmas;
    private double[][] thetas;

    public GaussianNB(double[] priors, double[][] sigmas, double[][] thetas) {
        this.priors = priors;
        this.sigmas = sigmas;
        this.thetas = thetas;
    }

    public int predict(double[] features) {
        double[] likelihoods = new double[this.sigmas.length];
    
        for (int i = 0, il = this.sigmas.length; i < il; i++) {
            double sum = 0.;
            for (int j = 0, jl = this.sigmas[0].length; j < jl; j++) {
                sum += Math.log(2. * Math.PI * this.sigmas[i][j]);
            }
            double nij = -0.5 * sum;
            sum = 0.;
            for (int j = 0, jl = this.sigmas[0].length; j < jl; j++) {
                sum += Math.pow(features[j] - this.thetas[i][j], 2.) / this.sigmas[i][j];
            }
            nij -= 0.5 * sum;
            likelihoods[i] = Math.log(this.priors[i]) + nij;
        }
    
        int cla